# **[Individual Assignment] Developing Spelling Corrector for Bahasa Indonesia**
- Pandya Athallah Erlambang 
- 19/444314/PA/*19376*

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
import re
from collections import Counter

def words(text): 
  return re.findall(r'\w+', text.lower())

Corpus = open('/content/drive/MyDrive/Colab Data/Spelling Corrector/ind_mixed_2012_300K/ind_mixed_2012_300K-sentences.txt').read()
WORDS = Counter(words(Corpora))

In [98]:
WORDS

Counter({'1': 4734,
         'kening': 52,
         'aie': 12,
         'yang': 151796,
         'terangkat': 86,
         'naik': 1289,
         'makin': 1140,
         'tinggi': 3010,
         '2': 4332,
         'bagaimana': 3300,
         'aktor': 150,
         'aktris': 34,
         'senior': 188,
         'dan': 109411,
         'punya': 3119,
         'bakat': 137,
         'akting': 24,
         'luarbiasa': 19,
         'bagus': 1001,
         'itu': 51588,
         'bisa': 18551,
         'diyakinkan': 13,
         'untuk': 42823,
         'tampil': 413,
         'superserius': 1,
         'atas': 6491,
         'skenario': 100,
         'sesungguhnya': 833,
         'tak': 12923,
         'serius': 538,
         '3': 3589,
         'nah': 739,
         'ingin': 4330,
         'tahu': 4524,
         'lebih': 13638,
         'detil': 55,
         'tentang': 6220,
         'stadium': 41,
         'aneka': 129,
         'ragam': 122,
         'istilah': 521,
         'perawatan'

## **Probability Theory**

The call correction(w) tries to choose the most likely spelling correction for w. There is no way to know for sure (for example, should "lates" be corrected to "late" or "latest" or "lattes" or ...?), which suggests we use probabilities. We are trying to find the correction c, out of all possible candidate corrections, that maximizes the probability that c is the intended correction, given the original word w:
> argmax c∈ candidates P(c|w)

By Bayes' Theorem this is equivalent to:
> argmax c∈ candidates P(c) P(w|c) / P(w)

Since P(w) is the same for every possible candidate c, we can factor it out, giving:
> argmax c∈ candidates P(c) P(w|c)

The four parts of this expression are:
1. **Selection Mechanism**: argmax
We choose the candidate with the highest combined probability.

2. **Candidate Model**: c∈ candidates
This tells us which candidate corrections, c, to consider.

3. **Language Model**: P(c)
The probability that c appears as a word of English text. For example, occurrences of "the" make up about 7% of English text, so we should have P(the) = 0.07.

4. **Error Model**: P(w|c)
The probability that w would be typed in a text when the author meant c. For example, P(teh|the) is relatively high, but P(theeexyz|the) would be very low.

One obvious question is: why take a simple expression like P(c|w) and replace it with a more complex expression involving two models rather than one? The answer is that P(c|w) is already conflating two factors, and it is easier to separate the two out and deal with them explicitly. Consider the misspelled word w="thew" and the two candidate corrections c="the" and c="thaw". Which has a higher P(c|w)? Well, "thaw" seems good because the only change is "a" to "e", which is a small change. On the other hand, "the" seems good because "the" is a very common word, and while adding a "w" seems like a larger, less probable change, perhaps the typist's finger slipped off the "e". The point is that to estimate P(c|w) we have to consider both the probability of c and the probability of the change from c to w anyway, so it is cleaner to formally separate the two factors.



## **Building the Language Model**

1. **Selection Mechanism**: In Python, max with a key argument does 'argmax'.


2. **Candidate Model**: First a new concept: a **simple edit** to a word is a deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter). The function edits1 returns a set of all the edited strings (whether words or not) that can be made with one simple edit:

In [99]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

This can be a big set. For a word of length n, there will be n deletions, n-1 transpositions, 26n alterations, and 26(n+1) insertions, for a total of 54n+25 (of which a few are typically duplicates). For example,

In [100]:
len(edits1('analsis'))

390

However, if we restrict ourselves to words that are known—that is, in the dictionary— then the set is much smaller:

In [101]:
def known(words): 
  return set(w for w in words if w in WORDS)

In [102]:
known(edits1('analsis'))

{'analis', 'analisis', 'analsis', 'analysis'}

We'll also consider corrections that require two simple edits. This generates a much bigger set of possibilities, but usually only a few of them are known words:

In [103]:
def edits2(word): 
  return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [104]:
len(set(edits2('analisis')))

90741

In [105]:
known(edits2('analisis'))

{'analiais',
 'analis',
 'analisa',
 'analisis',
 'analisisi',
 'analitik',
 'analitis',
 'analogis',
 'analsis',
 'analysis',
 'dianalisis',
 'paralisis'}

In [106]:
known(edits2('analsis'))

{'analaisa',
 'analiais',
 'analis',
 'analisa',
 'analisis',
 'analisisi',
 'analitis',
 'analogis',
 'analsis',
 'analysis',
 'anarkis',
 'anasir',
 'canalis',
 'kanalis',
 'narsis',
 'onasis'}

We say that the results of edits2(w) have an edit distance of 2 from w.

3. **Language Model**: We can estimate the probability of a word, P(word), by counting the number of times each word appears in a text file. The function words breaks text into words, then the variable WORDS holds a Counter of how often each word appears, and P estimates the probability of each word, based on this Counter:

3.1 **Unigram Model**

The unigram language model states that the probability of a sentence is determined by the product of the probability of each constituent word in the 
document.

- **P(w1w2...wn) = P(w1) x P(w2) x .... P(wn)**, where n is the number of constituent words in the sentence.
- P_uni is a function for computing the probability of each word in the document.

In [107]:
# Unigram model
def P_uni(word, N=sum(WORDS.values())): 
  return WORDS[word] / N

3.2 **Bigram Model**

The bigram language model follows the Markov's assumption, in which, the probability of a word depends only on the previous word.

- **P(wi|w1w2...w(i-1)) = P(wi|w(i-1))**, where i is the ith word in a sentence.
- freq is a function for finding the number of occurences of each possible consecutive word pairs in the document. This is required for calculating the conditional probability as the model is inspired from the Bayes Theorem
- P_bi is a similar function to P_uni, but it follows the bigram formula

In [108]:
def freq(input_string):
  freq = {}
  words = input_string.split()
  if len(words) == 1:
    return freq
  for idx, word in enumerate(words):
    if idx+1 < len(words):
      word_pair = (word, words[idx+1])
      if word_pair in freq:
        freq[word_pair] += 1
      else:
        freq[word_pair] = 1
  return freq

COUNT_PAIRS = freq(Corpus)
COUNT_PAIRS

{('1', '”Kening'): 1,
 ('”Kening', 'Aie'): 1,
 ('Aie', 'yang'): 2,
 ('yang', 'terangkat'): 4,
 ('terangkat', 'naik'): 1,
 ('naik', 'makin'): 1,
 ('makin', 'tinggi.'): 4,
 ('tinggi.', '2'): 1,
 ('2', 'Bagaimana'): 1,
 ('Bagaimana', 'aktor-aktris'): 1,
 ('aktor-aktris', 'senior'): 1,
 ('senior', 'dan'): 6,
 ('dan', 'yang'): 611,
 ('yang', 'punya'): 238,
 ('punya', 'bakat'): 10,
 ('bakat', 'akting'): 1,
 ('akting', 'luarbiasa'): 1,
 ('luarbiasa', 'bagus'): 1,
 ('bagus', 'itu'): 7,
 ('itu', 'bisa'): 401,
 ('bisa', 'diyakinkan'): 4,
 ('diyakinkan', 'untuk'): 2,
 ('untuk', 'tampil'): 28,
 ('tampil', 'superserius'): 1,
 ('superserius', 'atas'): 1,
 ('atas', 'skenario'): 1,
 ('skenario', 'yang'): 10,
 ('yang', 'sesungguhnya'): 100,
 ('sesungguhnya', 'tak'): 9,
 ('tak', 'serius?'): 1,
 ('serius?', '3'): 1,
 ('3', 'Nah,'): 1,
 ('Nah,', 'ingin'): 3,
 ('ingin', 'tahu'): 108,
 ('tahu', 'lebih'): 24,
 ('lebih', 'detil'): 4,
 ('detil', 'tentang'): 6,
 ('tentang', 'stadium'): 1,
 ('stadium', 'dan'): 2

In [109]:
# Bigram model
def P_bi(word, N=sum(COUNT_PAIRS.values())):
  sum = 0
  for pair_key in COUNT_PAIRS.keys():
    if word in pair_key:
      sum += COUNT_PAIRS[pair_key]
  return sum/N

We can see that there are 432,184 distinct words, which together appear 4,902,106 times, with 'yang' being the most common word, appearing 151,796 times (or a probability of about 3% using the unigram model and about 6% using the bigram model) and other words being less probable:

In [110]:
len(WORDS)

432184

In [111]:
sum(WORDS.values())

4902106

In [112]:
WORDS.most_common(10)

[('yang', 151796),
 ('dan', 109411),
 ('di', 70168),
 ('dengan', 54857),
 ('itu', 51588),
 ('ini', 44693),
 ('tidak', 43062),
 ('untuk', 42823),
 ('dari', 41478),
 ('dalam', 39131)]

In [113]:
print("Max P_uni:", max(WORDS, key = P_uni))
# print("Max P_bi:", max(WORDS, key = P_bi))

Max P_uni: yang


In [114]:
print("P_uni:", P_uni('yang'))
print("P_bi:", P_bi('yang'))

P_uni: 0.0309654666790151
P_bi: 0.06171189462504349


In [115]:
print("P_uni:", P_uni('konjungsi'))
print("P_bi:", P_bi('konjungsi'))

P_uni: 6.119818706490639e-07
P_bi: 4.152020414653975e-07


In [116]:
print("P_uni:", P_uni('sanubari'))
print("P_bi:", P_bi('sanubari'))

P_uni: 2.4479274825962556e-06
P_bi: 3.32161633172318e-06


4. **Error Model**: We can make candidates(word) produce the first non-empty list of candidates in order of priority:

> The original word, if it is known; otherwise

> The list of known words at edit distance one away, if there are any; otherwise

> The list of known words at edit distance two away, if there are any; otherwise

> The original word, even though it is not known.

Then we don't need to multiply by a P(w|c) factor, because every candidate at the chosen priority will have the same probability (according to our flawed model). That gives us:

In [117]:
def correction_uni(word): 
  return max(candidates(word), key = P_uni)

def correction_bi(word): 
  return max(candidates(word), key = P_bi)

def candidates(word): 
    return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]

The function correction(word) returns a likely spelling correction:

In [118]:
print(candidates('analosis'))
print("Unigram:", correction_uni('analosis'))
print("Bigram:", correction_bi('analosis'))

{'analsis', 'analisis', 'analysis', 'analogis'}
Unigram: analisis
Bigram: analisis


In [119]:
print(candidates('silakn'))
print("Unigram:", correction_uni('silakn'))
print("Bigram:", correction_bi('silakn'))

{'silakan', 'silaen'}
Unigram: silakan
Bigram: silakan


## **Evaluation**

In [120]:
def spelltest(tests, model, verbose = True):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        if model == 'uni':
          w = correction_uni(wrong)
        elif model == 'bi':
          w = correction_bi(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [121]:
# Unigram model test
spelltest(Testset(open('/content/drive/MyDrive/Colab Data/Spelling Corrector/error.txt')), 'uni')

correction(smeua) => smeua (1); expected ﻿semua (0)
correction(smua) => smua (22); expected ﻿semua (0)
correction(seuma) => semua (7573); expected ﻿semua (0)
correction(klas) => klas (25); expected kelas (1265)
correction(kela) => kela (1); expected kelas (1265)
correction(gaung) => gaung (8); expected gabung (23)
correction(gabun) => gaun (73); expected gabung (23)
correction(gbung) => bung (343); expected gabung (23)
correction(sekrang) => sekrang (5); expected sekarang (4690)
correction(skarang) => skarang (5); expected sekarang (4690)
correction(sekaran) => sekaran (1); expected sekarang (4690)
correction(seger) => seger (4); expected segar (266)
correction(sega) => sega (3); expected segar (266)
correction(sgar) => agar (4599); expected segar (266)
correction(untu) => untu (11); expected untuk (42823)
correction(utnuk) => utnuk (14); expected untuk (42823)
correction(tulikan) => tulisan (981); expected tuliskan (50)
correction(kerana) => kerana (34); expected karena (21065)
correc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


correction(perajurit) => perajurit (19); expected prajurit (619)
correction(pirarti) => piranti (34); expected prajurit (619)
correction(maslaha) => maslah (6); expected masalah (3138)
correction(malsahs) => malah (1026); expected masalah (3138)
correction(bearti) => bearti (4); expected berarti (2524)
correction(beariti) => bearti (4); expected berarti (2524)
correction(alaan) => alasan (1274); expected jalan (4627)
correction(ari) => ari (126); expected air (3336)
correction(baijr) => banjr (1); expected banjir (314)
correction(bantun) => bantun (1); expected bantuan (912)
correction(teapi) => teapi (3); expected tetapi (7326)
correction(ttpi) => tapi (10855); expected tetapi (7326)
correction(tapi) => tapi (10855); expected tetapi (7326)
correction(mdh) => sdh (59); expected mudah (1649)
correction(maduh) => madun (234); expected mudah (1649)
correction(tulung) => tulung (6); expected tolong (327)
correction(tulong) => tulong (1); expected tolong (327)
correction(hajun) => harun (60

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [122]:
# Bigram model test
spelltest(Testset(open('/content/drive/MyDrive/Colab Data/Spelling Corrector/error.txt')), 'bi')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


correction(smeua) => smeua (1); expected ﻿semua (0)
correction(smua) => smua (22); expected ﻿semua (0)
correction(seuma) => semua (7573); expected ﻿semua (0)
correction(klas) => klas (25); expected kelas (1265)
correction(kela) => kela (1); expected kelas (1265)
correction(gaung) => gaung (8); expected gabung (23)
correction(gabun) => gaun (73); expected gabung (23)
correction(sekrang) => sekrang (5); expected sekarang (4690)
correction(skarang) => skarang (5); expected sekarang (4690)
correction(sekaran) => sekaran (1); expected sekarang (4690)
correction(seger) => seger (4); expected segar (266)
correction(sega) => sega (3); expected segar (266)
correction(sgar) => agar (4599); expected segar (266)
correction(untu) => untu (11); expected untuk (42823)
correction(utnuk) => utnuk (14); expected untuk (42823)
correction(tulikan) => tulisan (981); expected tuliskan (50)
correction(kerana) => kerana (34); expected karena (21065)
correction(karna) => karna (56); expected karena (21065)
cor

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


## **Analysis**

From the evaluation above, we can see that both models have the same accuracy of 46% from 240 entries. Both models also have the same unknown words percentage of 1%. The differentiating factor thus lie in the processing speed of the models, where the unigram model is worlds faster with speed of 170 words/sec compared to the bigram model's mere 1 word/sec. Therefore we can conclude that for this use case, the unigram model is the better option, since it yields the same accuracy as the bigram model but at a much faster processing speed.

One problem worth noting for this assignment is that it seems that the corpus dataset contains non-baku Bahasa Indonesia words such as 'analisa' and 'jadual', English words such as 'drink' and 'drawing', as well as non-words such as 'banjr' and 'hannya'. This causes the model to yield undesired outputs. For example if the input is 'jadual', we expect the correction to be 'jadwal', but the output correction is still 'jadual' because indeed there is the word 'jadual' in the corpus and thus the model interpret the word to be already correct since it is in the corpus and made no further correction. A similar case also applies when the input word is 'darring'. We expect the correction to be 'daring' as in the abbreviation of 'dalam jaringan' or 'online', but the output correction is 'jarring' instead, an English word meaning 'incongruous in a striking or shocking way; clashing'. This occured because there are English words in our corpus, which there shouldn't be, and thus 'jarring' was prioritized more than 'daring', which is the correct output.

Also a minor problem is that the student-submitted error dataset contains error words that are actually correct words and are in the corpus, for instance 'dna' (as in the chemical molecule deoxyribonucleic acid) and 'gaung' (meaning echo in English). Those words were meant to be the misspelled form of 'dan' and 'gabung' respectively, but the student who submitted those words perhaps unknowingly wrote actual correct words. This also resulted in the models yielding an 'undesired' output as when we input the word 'dna' and 'gaung', it doesn't correct to 'dan' and 'gabung' because the former words are actually correct words that are in the corpus.

An explanation on how the unigram model is better than the bigram model might boil down to data sparsity: As our n-gram length increases, the amount of times we will see any given n-gram will decrease: In the most extreme example, if we have a corpus where the maximum document length is n tokens and you are looking for an m-gram where m=n+1, we will, of course, have no data points at all because it's simply not possible to have a sequence of that length in your data set. The more sparse your data set, the worse you can model it. For this reason, despite that a higher-order n-gram model, in theory, contains more information about a word's context, it cannot easily generalize to other data sets (known as overfitting) because the number of events (i.e. n-grams) it has seen during training becomes progressively less as n increases. On the other hand, a lower-order model lacks contextual information and so may underfit our data.

For this reason, if we have a very relatively large amount of token types (i.e. the vocabulary of your text is very rich) but each of these types has a very low frequency, we may get better results with a lower-order n-gram model. Similarly, if our training data set is very small, we may do better with a lower-order n-gram model. However, assuming that we have enough data to avoid over-fitting, we then get better separability of our data with a higher-order model.

Usually, n-grams more than one is better as it carries more information about the context in general. However, sometimes unigrams are also calculated besides bigram and trigrams and used as fallback for them. This is also useful if you want a higher recall than precision to search unigrams. For instance, we are searching for all possible uses of verb "make".

Lets use statistical machine translation as an example: intuitively, the best scenario is that our model has seen the full sentence (lets say 6-grams) before and knows its translation as a whole. If this is not the case we try to divide it to smaller n-grams, keeping into consideration that the more information we know about the word surroundings, the better the translation. For example, if we want to translate "Tom Green" to German, if we have seen the bi-gram we will know it is a person name and should remain as it is but if our model never saw it, you would fall back to unigrams and translate "Tom" and "Green" separately. Thus "Green" will be translated as a color to "Grün" and so on.

Also, in search knowing more about the surrounding context makes the results more accurate.
